In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark=SparkSession.builder.appName('SP').getOrCreate()

/home/labuser/PycharmProjects/ltiganesh1/venv/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
profit=spark.read.csv('/home/labuser/Downloads/movie/profit.csv',header=True)

In [4]:
profit.show()

+----------+-------------+
|movie_code| movie_profit|
+----------+-------------+
|       200|          120|
|       201|          140|
|       202|          190|
|       204|          150|
|       205|          110|
+----------+-------------+


In [5]:
details=spark.read.csv('/home/labuser/Downloads/movie/details.csv',header=True)

In [6]:
details.show()

+----------+----------+------------+
|movie_code|movie_name|movie_rating|
+----------+----------+------------+
|       200| shawshank|          95|
|       201|       saw|          85|
|       202|  avengers|          85|
|       204|        mi|          80|
|       205| wrongturn|         70 |
+----------+----------+------------+


In [7]:
# join the dataframes using the movie_name
joins=profit.join(details,on='movie_code',how='inner')

In [8]:
joins.show(3)

+----------+-------------+----------+------------+
|movie_code| movie_profit|movie_name|movie_rating|
+----------+-------------+----------+------------+
|       200|          120| shawshank|          95|
|       201|          140|       saw|          85|
|       202|          190|  avengers|          85|
+----------+-------------+----------+------------+


In [9]:
#top 3 for movie_profit and bottom 3 for movie_rating
joins.tail(3)

[Row(movie_code='202',  movie_profit='190', movie_name='avengers', movie_rating='85'),
 Row(movie_code='204',  movie_profit='150', movie_name='mi', movie_rating='80'),
 Row(movie_code='205',  movie_profit='110', movie_name='wrongturn', movie_rating='70 ')]

In [10]:
joins.orderBy(' movie_profit').show(3)

+----------+-------------+----------+------------+
|movie_code| movie_profit|movie_name|movie_rating|
+----------+-------------+----------+------------+
|       205|          110| wrongturn|         70 |
|       200|          120| shawshank|          95|
|       201|          140|       saw|          85|
+----------+-------------+----------+------------+


In [11]:
joins.orderBy(desc(' movie_profit')).show(3)

+----------+-------------+----------+------------+
|movie_code| movie_profit|movie_name|movie_rating|
+----------+-------------+----------+------------+
|       202|          190|  avengers|          85|
|       204|          150|        mi|          80|
|       201|          140|       saw|          85|
+----------+-------------+----------+------------+


In [12]:
joins.orderBy(['movie_rating']).show(3)

+----------+-------------+----------+------------+
|movie_code| movie_profit|movie_name|movie_rating|
+----------+-------------+----------+------------+
|       205|          110| wrongturn|         70 |
|       204|          150|        mi|          80|
|       201|          140|       saw|          85|
+----------+-------------+----------+------------+


In [13]:
#top 3 of profit to rating ratio

In [14]:
joins=joins.withColumn('ratio', round(joins[' movie_profit'] / joins['movie_rating'],2))

In [15]:
joins.show()

+----------+-------------+----------+------------+-----+
|movie_code| movie_profit|movie_name|movie_rating|ratio|
+----------+-------------+----------+------------+-----+
|       200|          120| shawshank|          95| 1.26|
|       201|          140|       saw|          85| 1.65|
|       202|          190|  avengers|          85| 2.24|
|       204|          150|        mi|          80| 1.88|
|       205|          110| wrongturn|         70 | 1.57|
+----------+-------------+----------+------------+-----+


In [16]:
joins.orderBy(desc('ratio')).show(3)

+----------+-------------+----------+------------+-----+
|movie_code| movie_profit|movie_name|movie_rating|ratio|
+----------+-------------+----------+------------+-----+
|       202|          190|  avengers|          85| 2.24|
|       204|          150|        mi|          80| 1.88|
|       201|          140|       saw|          85| 1.65|
+----------+-------------+----------+------------+-----+


In [17]:
# new question

In [18]:
# creating a new dataframe
df1=spark.read.csv('/home/labuser/Downloads/movie/seris.csv',header=True)

In [19]:
df1.show()

+---------+-----------+------------+
|series_id|series_info|     network|
+---------+-----------+------------+
|      101|       null|     netflix|
|      102|       null|amazon_prime|
|      103|       null|     appletv|
|      104|       null|     netflix|
|      105|       null|         HBO|
+---------+-----------+------------+


In [20]:
df2=spark.read.csv('/home/labuser/Downloads/movie/seris2.csv',header=True)

In [21]:
df2.show()

+---------+-------------+---------------+------+
|series_id|  series_name|          genre|rating|
+---------+-------------+---------------+------+
|      101| rickandmorty|          scifi|   9.1|
|      102|   invincible|       animated|   8.7|
|      103|forallmankind|historicalscifi|   7.9|
|      104|  blackmirror|  thrillerscifi|   8.0|
|      105|    westworld|          scifi|   8.1|
+---------+-------------+---------------+------+


In [22]:
ct=df2.select(concat_ws(' ',df2.series_name,df2.genre).alias('Fullname'))

In [23]:
ct.show()

+--------------------+
|            Fullname|
+--------------------+
|  rickandmorty scifi|
| invincible animated|
|forallmankind his...|
|blackmirror thril...|
|     westworld scifi|
+--------------------+


In [24]:
ct2=df2.select(concat_ws(' ',df2.series_name,df2.genre).alias('Fullname'),'series_id','rating')
ct2.show()

+--------------------+---------+------+
|            Fullname|series_id|rating|
+--------------------+---------+------+
|  rickandmorty scifi|      101|   9.1|
| invincible animated|      102|   8.7|
|forallmankind his...|      103|   7.9|
|blackmirror thril...|      104|   8.0|
|     westworld scifi|      105|   8.1|
+--------------------+---------+------+


In [25]:
df1.select(df1.series_info).show()

+-----------+
|series_info|
+-----------+
|       null|
|       null|
|       null|
|       null|
|       null|
+-----------+


In [26]:
df1.na.replace(df1.series_info,ct)

TypeError: to_replace should be a bool, float, int, string, list, tuple, or dict. Got <class 'pyspark.sql.column.Column'>

In [ ]:
j1=df1.join(ct2,on='series_id',how='inner')
j1.show()

In [ ]:
c1=j1.withColumn('series_info',coalesce(j1.series_info,j1.Fullname))

In [ ]:
c1.show()

In [ ]:
c2=c1.select(c1.series_id,c1.series_info,c1.network,c1.rating)
c2.show()